### Imports

In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from tqdm import tqdm
import string
import scipy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import HTML, display
from tqdm import tqdm
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordTokenizer()

[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
data_0 = pd.read_csv('../datasets/fossology-master-corrected.csv')
X_0 = data_0["copyright"]
y_0 = data_0["falsePositive"]
X_0 = X_0.drop_duplicates()
y_0 = y_0[X_0.index]

data_1 = pd.read_csv('../datasets/kubernetes-master-corrected.csv')
X_1 = data_1["copyright"]
y_1 = data_1["falsePositive"]
X_1 = X_1.drop_duplicates()
y_1 = y_1[X_1.index]

data_2 = pd.read_csv('../datasets/tensorflow-master-corrected.csv')
X_2 = data_2["copyright"]
y_2 = data_2["falsePositive"]
X_2 = X_2.drop_duplicates()
y_2 = y_2[X_2.index]

data_3 = pd.read_csv('../datasets/fossology-provided-1-corrected.csv')

X_3 = data_3['copyright']
y_3 = data_3['falsePositive']
X_3 = X_3.drop_duplicates()
y_3 = y_3[X_3.index]

data_4 = pd.read_csv('../datasets/fossology-provided-2.csv')

X_4 = data_4['copyright']
y_4 = data_4['falsePositive']
X_4 = X_4.drop_duplicates()
y_4 = y_4[X_4.index]

data_5 = pd.read_csv('../datasets/feature-extraction-paper.csv')

X_5 = data_5['copyright']
y_5 = data_5['falsePositive']
X_5 = X_5.drop_duplicates()
y_5 = y_5[X_5.index]

X = pd.concat([X_0, X_1, X_2, X_3])
y = pd.concat([y_0, y_1, y_2, y_3])

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

print('Class 0 Percentage: ', len(y[y == 0]) / len(y))
print('Class 1 Percentage: ', len(y[y == 1]) / len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Class 0 Percentage:  0.7522737712448323
Class 1 Percentage:  0.24772622875516767


In [4]:
def aggregate_reports(reports, print_aggregates=True):
    import pandas as pd
    import numpy as np
    dfs = []
    for metric in ['precision', 'recall', 'f1-score']:
        scores = []
        for report in reports:
            scores.append([report['0'][metric], report['1'][metric]])
        scores = np.array(scores)
        scores = scores[:, :2]
        mean_scores = np.mean(scores, axis=0)
        mean_scores = [f"{score:.6f}" for score in mean_scores]
        df = pd.DataFrame(scores, columns=['0', '1'])
        df.loc['Mean'] = mean_scores
        df['Metric'] = metric
        dfs.append(df)
    if print_aggregates:
        print("## Precision")
        print(dfs[0].to_markdown())
        print("## Recall")
        print(dfs[1].to_markdown())
        print("## F1-score")
        print(dfs[2].to_markdown())
    else:
        return dfs[0], dfs[1], dfs[2]

In [5]:
def get_missclassified_rows(X, y_true, y_pred, only_this_class = [0, 1], return_index=False):
    if type(y_true) != list:
        y_true = y_true.tolist()
    if type(y_pred) != list:
        y_pred = y_pred.tolist()
    if type(X) != list:
        X = X.tolist()
    missclassified_rows = []
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i] and y_true[i] in only_this_class:
            missclassified_rows.append(i)
    if return_index:
        return [(y_pred[i], i, X[i]) for i in missclassified_rows]
    else:
        return [(y_pred[i], X[i]) for i in missclassified_rows]

In [5]:
def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False, replace_names=False, replace_orgs=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if replace_names:
        pass
        if replace_orgs:
            pass
    if replace_orgs:
        if not replace_names:
            pass
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

### Start with testing out SpaCy one more time

In [7]:
nlp = spacy.load("en_core_web_sm")
processed_X = preprocess_function(X, replace_dates=True, remove_numbers=True)
processed_X_ents = [nlp(sentence) for sentence in tqdm(processed_X)]

100%|██████████| 21770/21770 [02:17<00:00, 157.99it/s]


In [9]:
spacy.displacy.serve(processed_X_ents[0:100], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [8]:
spacy.displacy.serve(processed_X_ents[200:300], style="ent", auto_select_port=True)

/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
spacy.displacy.serve(processed_X_ents[300:400], style="ent", auto_select_port=True)

### dataset - conll2003

In [14]:
!python -m spacy convert '../conll2003/train.txt' '../conll2003-spacy' -c ner
!python -m spacy convert '../conll2003/test.txt' '../conll2003-spacy' -c ner
!python -m spacy convert '../conll2003/valid.txt' '../conll2003-spacy' -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (14987 documents):
../conll2003-spacy/train.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3684 documents):
../conll2003-spacy/test.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3466 documents):
../conll2003-spacy/valid.spacy


In [35]:
!python -m spacy debug data '../conll2003-spacy/config.cfg'


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
14987 training docs
3466 evaluation docs
⚠ 130 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 204567 total word(s) in the data (23624 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 4 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
✔ No entities crossing sentence boundaries

================================== Summary ==================================
✔ 6 checks passed
⚠ 1 warning


In [36]:
!python -m spacy train '../conll2003-spacy/config.cfg' --output '../conll2003-spacy'

ℹ Saving to output directory: ../conll2003-spacy
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-08-29 16:29:50,333] [INFO] Set up nlp object from config
[2023-08-29 16:29:50,344] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-08-29 16:29:50,348] [INFO] Created vocabulary
[2023-08-29 16:29:50,349] [INFO] Finished initializing nlp object
[2023-08-29 16:30:02,532] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.06    0.97    1.03    0.93    0.01
  0     200        104.92   3350.78   53.54   55.06   52.10    0.54
  0     400        203.21   2182.20   69.05   69.71   68.39    0.69
  0     600        280.04   201

In [38]:
!python -m spacy evaluate '../conll2003-spacy/model-best' '../conll2003-spacy/test.spacy'

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   82.21
NER R   81.43
NER F   81.82
SPEED   28778


=============================== NER (per type) ===============================

           P       R       F
PER    84.38   81.82   83.08
LOC    85.91   88.07   86.97
ORG    78.09   76.82   77.45
MISC   77.86   75.64   76.73



In [6]:
nlp = spacy.load('../conll2003-spacy/model-best')
processed_X = preprocess_function(X, replace_dates=True, remove_numbers=True)
processed_X_ents = [nlp(sentence) for sentence in tqdm(processed_X)]

100%|██████████| 21770/21770 [00:56<00:00, 382.68it/s]


In [42]:
spacy.displacy.serve(processed_X_ents[200:300], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [43]:
spacy.displacy.serve(processed_X_ents[300:400], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [15]:
def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False, replace_names=False, replace_orgs=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False, replace_names_orgs=False, use_spacy_ner_model=False):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])
    if replace_names:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../conll2003-spacy/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' PERSON ', new_sentence)
                if replace_orgs:
                    if entity.label_ == 'ORG':
                        new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../conll2003-spacy/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_names_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../conll2003-spacy/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

In [14]:
# Train to save space in code
def train(svm, vectorizer, threshold, preprocess_function,**kwargs):
    X_train_tfidf = vectorizer.fit_transform(preprocess_function(X_train, **kwargs))
    X_test_tfidf = vectorizer.transform(preprocess_function(X_test, **kwargs))
    X_1_tfidf = vectorizer.transform(preprocess_function(X_1, **kwargs))
    X_2_tfidf = vectorizer.transform(preprocess_function(X_2, **kwargs))
    X_3_tfidf = vectorizer.transform(preprocess_function(X_3, **kwargs))
    preprocessed_X = preprocess_function(X, **kwargs)
    X_tfidf = vectorizer.transform(preprocessed_X)
    svm.fit(X_train_tfidf, y_train)
    if True: #svm.probability:
        y_pred = svm.predict_proba(X_test_tfidf)
        y_pred_1 = svm.predict_proba(X_1_tfidf)
        y_pred_2 = svm.predict_proba(X_2_tfidf)
        y_pred_3 = svm.predict_proba(X_3_tfidf)
        y_pred_4 = svm.predict_proba(X_tfidf)
        if threshold is None:
            y_pred_classification = np.argmax(y_pred, axis=1)
            y_pred_1_classification = np.argmax(y_pred_1, axis=1)
            y_pred_2_classification = np.argmax(y_pred_2, axis=1)
            y_pred_3_classification = np.argmax(y_pred_3, axis=1)
            y_pred_4_classification = np.argmax(y_pred_4, axis=1)
        else:
            y_pred_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred]
            y_pred_1_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_1]
            y_pred_2_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_2]
            y_pred_3_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_3]
            y_pred_4_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_4]
    else:
        y_pred_classification = svm.predict(X_test_tfidf)
        y_pred_1_classification = svm.predict(X_1_tfidf)
        y_pred_2_classification = svm.predict(X_2_tfidf)
        y_pred_3_classification = svm.predict(X_3_tfidf)
        y_pred_4_classification = svm.predict(X_tfidf)
    report = classification_report(y_test, y_pred_classification, output_dict=True)
    report_1 = classification_report(y_1, y_pred_1_classification, output_dict=True)
    report_2 = classification_report(y_2, y_pred_2_classification, output_dict=True)
    report_3 = classification_report(y_3, y_pred_3_classification, output_dict=True)
    report_4 = classification_report(y, y_pred_4_classification, output_dict=True)
    miss_classified_rows_0 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[0], return_index=True)
    miss_classified_rows_1 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[1], return_index=True)
    #aggregate_reports([report, report_1, report_2, report_3, report_4])
    print('Number of missclassifications in class 0: ', report_4['0']['support'] - round(report_4['0']['recall'] * report_4['0']['support']), 'out of a total sample of: ', report_4['0']['support'], ' - about ', round((1 - report_4['0']['recall']) * 100, 2), '% of the class was missclassified')
    print('Number of missclassifications in class 1: ', report_4['1']['support'] - round(report_4['1']['recall'] * report_4['1']['support']), 'out of a total sample of: ', report_4['1']['support'], ' - about ', round((1 - report_4['1']['recall']) * 100, 2), '% of the class was missclassified')
    return svm, vectorizer, miss_classified_rows_0, miss_classified_rows_1, preprocessed_X

In [81]:
processed_X_1 = preprocess_function(X, replace_dates=True, remove_numbers=True, replace_names=True)
processed_X_1[250]

'Copyright (C)  DATE - DATE  Siemens AG Author:  PERSON ,  PERSON ,  PERSON '

In [36]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
test_1 = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True)

Number of missclassifications in class 0:  27.0 out of a total sample of:  16377.0  - about  0.16 % of the class was missclassified
Number of missclassifications in class 1:  25.0 out of a total sample of:  5393.0  - about  0.46 % of the class was missclassified


In [16]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_names = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True)

Number of missclassifications in class 0:  23.0 out of a total sample of:  16377.0  - about  0.14 % of the class was missclassified
Number of missclassifications in class 1:  33.0 out of a total sample of:  5393.0  - about  0.61 % of the class was missclassified


In [17]:
only_names[2]

[(1, 2608, 'copyright eberhard mattes'),
 (1, 5474, 'copyright mark rodgers htm'),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the universal permissive license upl version '),
 (1,
  12160,
  'copyrighted by proprietary to and a trade secret of adaptec adaptec retains the title ownership and intellectual property rights in and to the software and all subsequent copies regardless of the form or media the software is protected by the copyright laws of the united states the european uni'),
 (1,
  13458,
  ' copyrightsymbol the european community date n nthis european union public person the eupl applies to the work or software as defined below which is provided under the terms of this person any use of the work other than as authorised under this person is prohibited to the extent such use is covered'),
 (1,
  14935,
  ' copyrightsymbol fossology contributors spdx license identifier gpl only file brief add the template path of decider twig templates to twig loader se

In [18]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_orgs = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_orgs=True)

Number of missclassifications in class 0:  33.0 out of a total sample of:  16377.0  - about  0.2 % of the class was missclassified
Number of missclassifications in class 1:  36.0 out of a total sample of:  5393.0  - about  0.67 % of the class was missclassified


In [19]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_orgs = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names_orgs=True)

Number of missclassifications in class 0:  28.0 out of a total sample of:  16377.0  - about  0.17 % of the class was missclassified
Number of missclassifications in class 1:  50.0 out of a total sample of:  5393.0  - about  0.93 % of the class was missclassified


In [21]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_names_spacy = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True, use_spacy_ner_model=True)

Number of missclassifications in class 0:  26.0 out of a total sample of:  16377.0  - about  0.16 % of the class was missclassified
Number of missclassifications in class 1:  25.0 out of a total sample of:  5393.0  - about  0.46 % of the class was missclassified


In [110]:
only_names_spacy[2]

[(1, 2608, 'copyright eberhard mattes'),
 (1, 2794, 'copyright richard thomson'),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the universal permissive license upl version '),
 (1,
  12160,
  'copyrighted by proprietary to and a trade secret of adaptec adaptec retains the title ownership and intellectual property rights in and to the software and all subsequent copies regardless of the form or media the software is protected by the copyright laws of the united states the european uni'),
 (1,
  13373,
  'copyright date date date by paul vixie n nall rights reserved n ndistribute freely except don t remove my name from the source or documentation don t take credit for my work mark your changes don t get me blamed for your possible bugs don t alter or remove this notice may be sold if buildab'),
 (1,
  13458,
  ' copyrightsymbol the european community date n nthis european union public licence the eupl applies to the work or software as defined below which is provided

In [23]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_orgs_spacy = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_orgs=True, use_spacy_ner_model=True)

  3%|▎         | 595/21770 [45:22<26:54:45,  4.58s/it]


Number of missclassifications in class 0:  35.0 out of a total sample of:  16377.0  - about  0.21 % of the class was missclassified
Number of missclassifications in class 1:  32.0 out of a total sample of:  5393.0  - about  0.59 % of the class was missclassified


In [ ]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
names_orgs_spacy = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names_orgs=True, use_spacy_ner_model=True)

In [24]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_names = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True)

Number of missclassifications in class 0:  3.0 out of a total sample of:  16377.0  - about  0.02 % of the class was missclassified
Number of missclassifications in class 1:  241.0 out of a total sample of:  5393.0  - about  4.47 % of the class was missclassified


In [113]:
only_names[2]

[(1, 2608, 'copyright eberhard mattes'),
 (1,
  13458,
  ' copyrightsymbol the european community date n nthis european union public person the eupl applies to the work or software as defined below which is provided under the terms of this person any use of the work other than as authorised under this person is prohibited to the extent such use is covered'),
 (1, 17694, 'copyright copyrightsymbol all rights reserved ')]

### Fine tuning the best model as well as en_core_web_sm on our dataset (750 rows for a start)

### Using Doccano along with SpaCy to annotate this dataset

In [12]:
import json

nlp = spacy.load('../conll2003-spacy/model-best')
X_ner = [nlp(sentence) for sentence in tqdm(X)]

def text_to_doccano(sentences):
    """
    :text (str): source text
    Returns (list (dict)): deccano format json
    """
    djson = list()
    #doc = nlp(text)
    for sentence in tqdm(sentences):
        labels = list()
        for e in sentence.ents:
            labels.append([e.start_char, e.end_char, e.label_])
        djson.append({'text': sentence.text, "labels": labels})
    return djson

doccano_json = text_to_doccano(X_ner)
with open('dataset-ner.jsonl', 'w') as f:
    for item in doccano_json:
        f.write(json.dumps(item) + '\n')

100%|██████████| 21770/21770 [00:00<00:00, 57443.42it/s]


### Try training spacy on conll2003 from scratch

In [ ]:
import json
import spacy
from spacy.tokens import Doc, DocBin
from spacy.training import Example
from spacy.util import minibatch

def load_data(file_path):
    """
    Load JSONL data from a file and convert it to spaCy's training format.
    :param file_path: The path of the JSONL file.
    :return: A list of training examples.
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            item = json.loads(line)
            text = item['text']
            entities = [(e[0], e[1], e[2]) for e in item['label']]
            data.append((text, {'entities': entities}))
    return data

def save_data(data, file_path):
    """
    Save data in spaCy's binary format.
    :param data: The training data.
    :param file_path: The path of the output file.
    """
    nlp = spacy.blank('en')
    doc_bin = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        doc_bin.add(example.reference)
    doc_bin.to_disk(file_path)

# Load the data
data = load_data('../Fossology-NER-dataset-creation/1-750.jsonl')

# Save the data
save_data(data, '../Fossology-NER-dataset-creation/1-750.spacy')


In [36]:
!python -m spacy debug data '../Fossology-NER-dataset-creation/config.cfg'


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
750 training docs
750 evaluation docs
⚠ 750 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (750)

============================== Vocab & Vectors ==============================
ℹ 9178 total word(s) in the data (1504 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 3 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
✔ No entities crossing sentence boundaries

================================== Summary ==================================
✔ 6 chec

In [37]:
!python -m spacy train '../Fossology-NER-dataset-creation/config.cfg' --output '../Fossology-NER-dataset-creation' 

ℹ Saving to output directory: ../Fossology-NER-dataset-creation
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-08-30 12:34:10,756] [INFO] Set up nlp object from config
[2023-08-30 12:34:10,767] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-08-30 12:34:10,771] [INFO] Created vocabulary
[2023-08-30 12:34:10,772] [INFO] Finished initializing nlp object
[2023-08-30 12:34:11,183] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     51.43    0.00    0.00    0.00    0.00
  2     200         84.31   1950.63   94.73   95.64   93.84    0.95
  4     400        108.90    318.26   98.11   98.40   97.81    0.98
  8     600     

In [38]:
!python -m spacy evaluate '../Fossology-NER-dataset-creation/model-best' '../conll2003-spacy/test.spacy'

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   41.81
NER R   24.93
NER F   31.23
SPEED   27214


=============================== NER (per type) ===============================

           P       R       F
PER    47.92   56.22   51.74
LOC    47.15   23.80   31.63
MISC    0.00    0.00    0.00
ORG    16.22    6.14    8.91



In [39]:
def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False, replace_names=False, replace_orgs=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False, replace_names_orgs=False, use_spacy_ner_model=False):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])
    if replace_names:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' PERSON ', new_sentence)
                if replace_orgs:
                    if entity.label_ == 'ORG':
                        new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_names_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

In [40]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_names_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True)

Number of missclassifications in class 0:  22.0 out of a total sample of:  16377.0  - about  0.13 % of the class was missclassified
Number of missclassifications in class 1:  36.0 out of a total sample of:  5393.0  - about  0.67 % of the class was missclassified


In [41]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
only_orgs_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_orgs=True)

Number of missclassifications in class 0:  26.0 out of a total sample of:  16377.0  - about  0.16 % of the class was missclassified
Number of missclassifications in class 1:  27.0 out of a total sample of:  5393.0  - about  0.5 % of the class was missclassified


In [42]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
names_orgs_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True, replace_orgs=True)

Number of missclassifications in class 0:  21.0 out of a total sample of:  16377.0  - about  0.13 % of the class was missclassified
Number of missclassifications in class 1:  37.0 out of a total sample of:  5393.0  - about  0.69 % of the class was missclassified


In [43]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
names_orgs_fine_combined_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names_orgs=True)

Number of missclassifications in class 0:  19.0 out of a total sample of:  16377.0  - about  0.12 % of the class was missclassified
Number of missclassifications in class 1:  38.0 out of a total sample of:  5393.0  - about  0.7 % of the class was missclassified


### Compare with fine tuning the original spacy model trained on ontoshot (colln2012)

In [45]:
nlp = spacy.load('en_core_web_sm')
weights = nlp.get_pipe('tok2vec').model.to_bytes()
with open('../Fossology-NER-dataset-creation/en_core_web_sm/tok2vec_weights', 'wb') as f:
    f.write(weights)

In [46]:
!python -m spacy train '../Fossology-NER-dataset-creation/en_core_web_sm/config.cfg' --output '../Fossology-NER-dataset-creation/en_core_web_sm' 

ℹ Saving to output directory:
../Fossology-NER-dataset-creation/en_core_web_sm
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-08-30 13:11:16,766] [INFO] Set up nlp object from config
[2023-08-30 13:11:16,779] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-08-30 13:11:16,782] [INFO] Created vocabulary
[2023-08-30 13:11:16,783] [INFO] Finished initializing nlp object
[2023-08-30 13:11:17,201] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     51.43    0.00    0.00    0.00    0.00
  2     200         84.31   1950.63   94.73   95.64   93.84    0.95
  4     400        108.90    318.26   98.11   98.40   97.81    0.98
 

In [47]:
def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False, replace_names=False, replace_orgs=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False, replace_names_orgs=False, use_spacy_ner_model=False):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])
    if replace_names:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/en_core_web_sm/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' PERSON ', new_sentence)
                if replace_orgs:
                    if entity.label_ == 'ORG':
                        new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/en_core_web_sm/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ORG ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_names_orgs:
        if use_spacy_ner_model:
            nlp = spacy.load('en_core_web_sm', disable=["parser", "tagger"])
        else:            
            nlp = spacy.load('../Fossology-NER-dataset-creation/en_core_web_sm/model-best')
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'PER':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
                if entity.label_ == 'ORG':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

In [48]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
spacy_only_names_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True)

Number of missclassifications in class 0:  21.0 out of a total sample of:  16377.0  - about  0.13 % of the class was missclassified
Number of missclassifications in class 1:  36.0 out of a total sample of:  5393.0  - about  0.67 % of the class was missclassified


In [ ]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
spacy_only_orgs_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_orgs=True)

In [ ]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
spacy_names_orgs_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names=True, replace_orgs=True)

In [49]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
spacy_names_orgs_combined_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names_orgs=True)

Number of missclassifications in class 0:  20.0 out of a total sample of:  16377.0  - about  0.12 % of the class was missclassified
Number of missclassifications in class 1:  38.0 out of a total sample of:  5393.0  - about  0.7 % of the class was missclassified


In [59]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
spacy_names_orgs_combined_fine_tuned = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_names_orgs=True)

Number of missclassifications in class 0:  4.0 out of a total sample of:  16377.0  - about  0.02 % of the class was missclassified
Number of missclassifications in class 1:  312.0 out of a total sample of:  5393.0  - about  5.79 % of the class was missclassified


In [61]:
spacy_names_orgs_combined_fine_tuned[2]

[(1,
  7437,
  'copyright notice a class top href releaseheader date a h pre class licensetext id licensetext entity grants reproduction and use rights to all parties provided that this comment is maintained in the copy pre li '),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the universal permissive license entity version '),
 (1,
  13458,
  ' copyrightsymbol the entity date n nthis entity entity the eupl applies to the work or software as defined below which is provided under the terms of this licence any use of the work other than as authorised under this licence is prohibited to the extent such use is covered'),
 (1, 17694, 'copyright copyrightsymbol all rights reserved ')]

In [62]:
X[13458]

'(c) the European Community 2007\\n\\nThis European Union Public Licence (the \\"EUPL\\") applies to the Work or Software (as defined below) which is provided under the terms of this Licence. Any use of the Work, other than as authorised under this Licence is prohibited (to the extent such use is covered'

In [58]:
X[7327]

'Copyright (C) YEAR Free Software Foundation, Inc.'

In [56]:
spacy_names_orgs_combined_fine_tuned[2]

[(1, 5474, 'copyright mark rodgers htm'),
 (1, 7327, 'copyright copyrightsymbol year entity '),
 (1,
  7437,
  'copyright notice a class top href releaseheader date a h pre class licensetext id licensetext entity grants reproduction and use rights to all parties provided that this comment is maintained in the copy pre li '),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the universal permissive license entity version '),
 (1,
  12160,
  'copyrighted by proprietary to and a trade secret of adaptec adaptec retains the title ownership and intellectual property rights in and to the software and all subsequent copies regardless of the form or media the software is protected by the copyright laws of the entity the entity '),
 (1,
  13458,
  ' copyrightsymbol the entity date n nthis entity entity the eupl applies to the work or software as defined below which is provided under the terms of this licence any use of the work other than as authorised under this licence is proh

### Let's visually checkout the NER performance

In [50]:
nlp = spacy.load("../Fossology-NER-dataset-creation/en_core_web_sm/model-best")
X_ner_spacy_finetuned = [nlp(sentence) for sentence in tqdm(X)]

100%|██████████| 21770/21770 [01:02<00:00, 346.01it/s]


In [52]:
spacy.displacy.serve(X_ner_spacy_finetuned[0:100], style="ent", auto_select_port=True)

/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [53]:
spacy.displacy.serve(X_ner_spacy_finetuned[200:300], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [54]:
spacy.displacy.serve(X_ner_spacy_finetuned[300:400], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [55]:
spacy.displacy.serve(X_ner_spacy_finetuned[12000:12100], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


#############################

In [ ]:


dataset = load_dataset("conll2012_ontonotesv5", 'english_v12')

def preprocess(example):
    tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
    
    encodings = []
    for sublist in example["sentences"]["words"]:
        encoding = tokenizer(sublist, padding=True, truncation=True, return_tensors="pt")
        encodings.append(encoding)
    
    labels = example["sentences"]["named_entities"]  # Assuming each sublist has corresponding named entities
    
    return [{"input_ids": e["input_ids"].squeeze(), "attention_mask": e["attention_mask"].squeeze(), "labels": l}
            for e, l in zip(encodings, labels)]

dataset = dataset.map(preprocess)

train_loader = DataLoader(dataset["train"], batch_size=32, shuffle=True)
val_loader = DataLoader(dataset["validation"], batch_size=32)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'sentences'],
        num_rows: 10539
    })
    validation: Dataset({
        features: ['document_id', 'sentences'],
        num_rows: 1370
    })
    test: Dataset({
        features: ['document_id', 'sentences'],
        num_rows: 1200
    })
})

In [ ]:
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=37)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

criterion = torch.nn.CrossEntropyLoss()

epochs = 20

for epoch in range(epochs):
  model.train()
  train_loss = 0.0
  train_acc = 0.0
  # Loop over the training batches
  for batch in train_loader:
    # Move the batch tensors to GPU if available
    batch = {k: v.to(device) for k, v in batch.items()}
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    output = model(**batch)
    # Compute the loss
    loss = criterion(output.logits, batch["labels"])
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    # Update the training loss and accuracy
    train_loss += loss.item()
    train_acc += (output.logits.argmax(dim=1) == batch["labels"]).float().mean().item()
  # Compute the average training loss and accuracy
  train_loss /= len(train_loader)
  train_acc /= len(train_loader)
  # Update the learning rate scheduler
  scheduler.step()
  # Set the model to evaluation mode
  model.eval()
  # Initialize the validation loss and accuracy
  val_loss = 0.0
  val_acc = 0.0
  # Loop over the validation batches
  with torch.no_grad():
    for batch in val_loader:
      # Move the batch tensors to GPU if available
      batch = {k: v.to(device) for k, v in batch.items()}
      # Forward pass
      output = model(**batch)
      # Compute the loss
      loss = criterion(output.logits, batch["labels"])
      # Update the validation loss and accuracy
      val_loss += loss.item()
      val_acc += (output.logits.argmax(dim=1) == batch["labels"]).float().mean().item()
  # Compute the average validation loss and accuracy
  val_loss /= len(val_loader)
  val_acc /= len(val_loader)
  # Print the epoch summary
  print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
